Importing Necessary Libraries

In [11]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install tensorflow



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install setuptools


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Creating Static Guesture dataset

In [ ]:
gestures=["open_palm","thumbs_up","thumbs_down","victory","fist","spiderman","fuck_off"] #Static Guestures

In [ ]:
#One-by-one capture the datasets and save them.

import cv2
import os
import time

gesture_name="fuck_off" 
def capture_images(gesture_name, num_images=500, delay=0.2):
    save_path = f"data/static/{gesture_name}"
    os.makedirs(save_path, exist_ok=True)

    cap = cv2.VideoCapture(0)  # Open webcam
    count = 0

    while count < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        # Flip the image to create the inverted version
        flipped_frame = cv2.flip(frame, 1)

        # Display the live feed
        cv2.putText(frame, f"Capturing {gesture_name} ({count}/{num_images})",
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow("Capture Gesture", frame)

        # Save both original and flipped images
        cv2.imwrite(f"{save_path}/{count}.jpg", frame)  # Original
        cv2.imwrite(f"{save_path}/{count}_flipped.jpg", flipped_frame)  # Flipped version

        count += 1
        time.sleep(delay)  # Delay between captures

        # Press 'q' to stop early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the function
capture_images(gesture_name, num_images=500, delay=0.2)

Hand landmark using media pipe

In [ ]:
#Extracting the landmarks from the images and saving them to a CSV file.

import cv2
import mediapipe as mp
import numpy as np
import csv
import os

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Define dataset path and CSV file
dataset_path = "C:\\Users\\AYUSH\\OneDrive\\Desktop\\CPP\\Machine Learning\\data\\static"

csv_file = "hand_landmarks.csv"

# Create CSV file and write header
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    header = ["gesture"] + [f"x{i},y{i},z{i}" for i in range(21)]  # 21 landmarks
    writer.writerow(header)

# Process each gesture folder
for gesture_name in os.listdir(dataset_path):
    gesture_path = os.path.join(dataset_path, gesture_name)
    
    if not os.path.isdir(gesture_path):
        continue

    for img_name in os.listdir(gesture_path):
        img_path = os.path.join(gesture_path, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        # Convert to RGB and process with Mediapipe
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_image)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract landmarks
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])  # Flatten (x, y, z)

                # Save to CSV
                with open(csv_file, mode='a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([gesture_name] + landmarks)

print("✅ Hand landmarks extracted and saved to CSV!")


✅ Hand landmarks extracted and saved to CSV!


In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("hand_landmarks.csv")

# Separate features (landmarks) and labels (gesture names)
X = df.iloc[:, 1:].values  # Hand landmarks (exclude gesture column)
y = df.iloc[:, 0].values  # Gesture names

# Convert gesture labels to numbers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#np.save("label_classes.npy", label_encoder.classes_)

print("✅ Data Loaded and Preprocessed!")
print(f"Total Samples: {len(df)}, Training: {len(X_train)}, Validation: {len(X_val)}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

✅ Data Loaded and Preprocessed!
Total Samples: 5718, Training: 4574, Validation: 1144
X_train shape: (4574, 21), y_train shape: (4574,)


Train a neural network to classify the gestures.

In [ ]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Define the model
model = Sequential([
    
    Dense(128, activation='relu', input_shape=(63,)),
    BatchNormalization(),
    #Dropout(0.5),

    Dense(64, activation='relu'),
    BatchNormalization(),
    #Dropout(0.2),

    Dense(32, activation='relu'),
    #Dropout(0.2),

    Dense(7, activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print(f'Shape of y_train: {y_train.shape}, Shape of y_val: {y_val.shape}')

# Train the model (Put the callback inside fit)
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_val, y_val), 
                    callbacks=[lr_scheduler])


# Save the trained model
model.save("gesture_model.keras")

print("✅ Model Trained and Saved!")

Shape of y_train: (4574, 5718), Shape of y_val: (1144, 5717)


c:\Users\AYUSH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 63, but received input with shape (None, 21)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 21), dtype=float32)
  • training=True
  • mask=None

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,672 (151.07 KB)

 Trainable params: 19,143 (74.78 KB)

 Non-trainable params: 384 (1.50 KB)

 Optimizer params: 19,145 (74.79 KB)

Function to determine if Static or Dynamic movement

In [ ]:
import numpy as np
def isStatic(landmark_history,threshold=10):
    if len(landmark_history)<2:
        return True
    distance=np.mean(np.linalg.norm(np.array(landmark_history[-1])-np.array(landmark_history[-2])))
    return distance<threshold

In [ ]:
#Script for Static guesture prediction

import numpy as np
import cv2
import mediapipe as mp
from tensorflow import keras  
from collections import deque
from sklearn.preprocessing import LabelEncoder

# Load trained model and label encoder
model = keras.models.load_model("gesture_model.keras")
label_encoder = LabelEncoder()
label_classes = np.load("label_classes.npy", allow_pickle=True)
label_encoder.classes_ = label_classes  # Assign loaded labels

# Debug: Print classes to verify correct loading
print("Loaded gesture classes:", label_classes)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Store last 30 frames of landmarks
landmark_history = deque(maxlen=30)

# Function to detect if hand is static
def isStatic(landmark_history, threshold=0.1):  # Lowered threshold for better sensitivity
    """Check if hand is static by comparing last two frames."""
    if len(landmark_history) < 2:
        return True  # Assume static if not enough frames

    # Compute Euclidean distance between last two frames
    distance = np.linalg.norm(np.array(landmark_history[-1]) - np.array(landmark_history[-2]))

    return distance < threshold  # If movement is below threshold, it's static

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract landmarks (21 points, each with x, y, z)
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            # Store in history
            landmark_history.append(landmarks)

            # Check if static or dynamic
            if isStatic(landmark_history):
                if len(landmarks) == 63:  # Ensure correct shape
                    landmarks_array = np.array(landmarks).reshape(1, -1).astype(np.float32)

                    # Make prediction
                    prediction = model.predict(landmarks_array)
                    print("Raw model output:", prediction)  # Print raw model output

                    # Convert logits to probabilities (only if your model doesn't already output probabilities)
                    if prediction.shape[1] > 1:  # Check if multi-class
                        prediction_prob = keras.activations.softmax(keras.backend.variable(prediction)).numpy()
                    else:
                        prediction_prob = prediction  # If already probabilities

                    print("Softmax probabilities:", prediction_prob)  # Debug print

                    # Get predicted class index
                    predicted_index = np.argmax(prediction_prob)
                    print("Predicted index:", predicted_index)  # Debug print

                    # Ensure the index is valid
                    if 0 <= predicted_index < len(label_encoder.classes_):
                        gesture = label_encoder.inverse_transform([predicted_index])[0]
                    else:
                        gesture = "Unknown"

                    print("Predicted gesture:", gesture)  # Debug print


                    # Ensure valid index
                    if 0 <= predicted_index < len(label_encoder.classes_):
                        if(predicted_index==0):
                            gesture="fist"
                        if(predicted_index==1):
                            gesture="fuck off"
                        if(predicted_index==2):
                            gesture="open palm" 
                        if(predicted_index==3):
                            gesture="spiderman"
                        if(predicted_index==5):
                            gesture="thumbs upp"
                        if(predicted_index==6):
                            gesture="thumbs down"
                        if(predicted_index==6):
                            gesture="victory"  
                        #gesture = label_encoder.inverse_transform([predicted_index])[0]
                    else:
                        gesture = "Unknown"

                    # Display static gesture
                    cv2.putText(frame, "Static", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(frame, f"Gesture: {gesture}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "Error: Landmark Shape", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Dynamic", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Hand Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()


Loaded gesture classes: [-0.2371041  -0.22068678 -0.21517485 ...  0.1248069   0.13180147
  0.13476376]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Raw model output: [[1.3891624e-02 2.0795469e-03 9.7596258e-01 1.7071060e-03 5.5302093e-03
  2.1913486e-06 8.2672562e-04]]
Softmax probabilities: [[0.11684758 0.11547548 0.30580303 0.11543248 0.11587464 0.11523585
  0.1153309 ]]
Predicted index: 2
Predicted gesture: -0.2151748538017273
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Raw model output: [[3.6266844e-08 3.1929314e-06 9.9999666e-01 4.8083901e-08 1.4639498e-08
  8.9282054e-11 1.2173392e-07]]
Softmax probabilities: [[0.11470158 0.11470195 0.31179014 0.11470158 0.11470158 0.11470158
  0.11470159]]
Predicted index: 2
Predicted gesture: -0.2151748538017273
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Raw model output: [[8.4088754e-08 3.1937807e-06 9.9999654e-01 5.2712814e-08 1.6675825e-09
  7.1568203e-11 7.9420573e-08]]
Softmax probabilities: [[0.11470158 0.11470194 0.3117901  0.11470158 0.11470158 0.114701

Dynamic data set

In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
import pandas as pd

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Set storage paths
DATASET_PATH = "dataset_dymnamic"
CSV_FILE = "hand_landmarks_dynamic.csv"

# Create dataset folder if not exists
os.makedirs(DATASET_PATH, exist_ok=True)

# Initialize CSV storage
columns = ["frame_number", "gesture"] + [f"{axis}{i}" for i in range(21) for axis in ["x", "y", "z"]]
if not os.path.exists(CSV_FILE):
    pd.DataFrame(columns=columns).to_csv(CSV_FILE, index=False)

# Get gesture name from user
s="Bye"
gesture_name = s.strip().lower()
gesture_folder = os.path.join(DATASET_PATH, gesture_name)
os.makedirs(gesture_folder, exist_ok=True)

# Open webcam
cap = cv2.VideoCapture(0)
frame_count = 0
landmark_data = []

print("Press 'q' to stop recording.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    frame_path = os.path.join(gesture_folder, f"{frame_count}.jpg")
    cv2.imwrite(frame_path, frame)  # Save the RGB frame

    # Process hand landmarks
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract 21 hand landmarks
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            # Save landmarks to list
            landmark_data.append([frame_count, gesture_name] + landmarks)

    cv2.imshow("Recording Gesture", frame)
    frame_count += 1

    # Stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Append landmark data to CSV
df = pd.DataFrame(landmark_data, columns=columns)
df.to_csv(CSV_FILE, mode='a', header=False, index=False)

print(f"✅ Gesture '{gesture_name}' recorded successfully!")


Press 'q' to stop recording.


KeyboardInterrupt: 

Change the guesture name and csv file name

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd

# Configuration
GESTURE_NAME = "M Chuda"  # Change this for different gestures
SAVE_DIR = "dataset"
FRAME_LIMIT = 3000  # Total frames to capture
BATCH_SIZE = 30  # Frames per sample
SAMPLES = 100  # Total gesture samples

# Create directories
rgb_path = os.path.join(SAVE_DIR, GESTURE_NAME, "RGB_Frames")
os.makedirs(rgb_path, exist_ok=True)

# Initialize Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Start Video Capture
cap = cv2.VideoCapture(0)

frames = []
landmarks_list = []

print(f"Recording gesture: {GESTURE_NAME}...")

frame_count = 0
while frame_count < FRAME_LIMIT:
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # Save RGB frame
    frame_filename = os.path.join(rgb_path, f"frame_{frame_count}.jpg")
    cv2.imwrite(frame_filename, frame)

    # Extract landmarks
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lm_list = []
            for lm in hand_landmarks.landmark:
                lm_list.extend([lm.x, lm.y, lm.z])
            landmarks_list.append([frame_count] + lm_list)  # Add frame index
    else:
        landmarks_list.append([frame_count] + [None] * 63)  # Empty if no hand detected

    # Display recording progress
    cv2.putText(frame, f"Recording {frame_count}/{FRAME_LIMIT}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Recording", frame)

    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit early
        break

cap.release()
cv2.destroyAllWindows()

# Save all landmarks in one CSV file
csv_path = os.path.join(SAVE_DIR, GESTURE_NAME, "landmarks_4.csv")
columns = ["Frame"] + [f"LM_{i}" for i in range(63)]  # Column names
landmarks_df = pd.DataFrame(landmarks_list, columns=columns)
landmarks_df.to_csv(csv_path, index=False)

print("✅ Recording Complete!")

# --- Batching into 100 samples ---
print("Batching frames into 100 samples with mirrored versions...")

# Ensure correct number of frames
if len(landmarks_list) >= FRAME_LIMIT:
    for sample_idx in range(SAMPLES):
        start_idx = sample_idx * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE

        if end_idx <= FRAME_LIMIT:
            batch_rgb_dir = os.path.join(rgb_path, f"sample_{sample_idx}")
            os.makedirs(batch_rgb_dir, exist_ok=True)

            # Save RGB frames for this batch
            for i in range(start_idx, end_idx):
                original_path = os.path.join(rgb_path, f"frame_{i}.jpg")
                mirrored_path = os.path.join(batch_rgb_dir, f"mirrored_{i % BATCH_SIZE}.jpg")
                new_path = os.path.join(batch_rgb_dir, f"frame_{i % BATCH_SIZE}.jpg")

                # Move original
                os.rename(original_path, new_path)

                # Create and save mirrored image
                img = cv2.imread(new_path)
                mirrored_img = cv2.flip(img, 1)  # Horizontal flip
                cv2.imwrite(mirrored_path, mirrored_img)

print("✅ Batching Complete!")

Recording gesture: M Chuda...
✅ Recording Complete!
Batching frames into 100 samples with mirrored versions...
✅ Batching Complete!


In [60]:
import numpy as np

# Load data first
X_train1 = np.load("X_train.npy")
X_test1 = np.load("X_test.npy")
y_train1 = np.load("y_train.npy")
y_test1 = np.load("y_test.npy")

# Keep only the first 63 features (21 landmarks * 3 coordinates)
X_train1 = X_train[:, :, :63]
X_test1 = X_test1[:, :, :63]

# Print shapes for verification
print(f"X_train shape after slicing: {X_train1.shape}")  # Expected: (num_samples, 30, 63)
print(f"X_test shape after slicing: {X_test1.shape}")
print(f"y_train shape: {y_train1.shape}")
print(f"y_test shape: {y_test1.shape}")


X_train shape after slicing: (320, 30, 63)
X_test shape after slicing: (80, 30, 63)
y_train shape: (320, 4)
y_test shape: (80, 4)


In [54]:
print(X_train.shape)
X_train[0,0]

(320, 30, 63)


array([ 9.00000000e+01,  5.31555951e-01,  9.12091613e-01, -2.50735297e-07,
        4.76505399e-01,  8.43463778e-01, -1.33538423e-02,  4.52662855e-01,
        7.38568306e-01, -2.93067899e-02,  4.66599166e-01,  6.41250968e-01,
       -4.09852825e-02,  5.06354570e-01,  5.97764015e-01, -4.94472235e-02,
        4.82463092e-01,  7.78885663e-01, -6.65162951e-02,  5.07402778e-01,
        6.20306313e-01, -9.62230638e-02,  4.98349488e-01,  6.11827910e-01,
       -1.00337245e-01,  4.90918249e-01,  6.39743865e-01, -9.57352221e-02,
        5.41257262e-01,  7.91390777e-01, -6.67643920e-02,  5.50480008e-01,
        6.20874524e-01, -9.30399373e-02,  5.32759428e-01,  6.38566732e-01,
       -8.64594132e-02,  5.27896464e-01,  6.83886051e-01, -7.50860572e-02,
        5.93722522e-01,  7.95771897e-01, -6.50351122e-02,  5.88451743e-01,
        6.42645299e-01, -8.36455300e-02,  5.67662776e-01,  6.69328392e-01,
       -6.63551837e-02,  5.63294172e-01,  7.13357210e-01, -4.86729443e-02,
        6.36539519e-01,  

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Dataset path
DATASET_PATH = "dataset"
NUM_FRAMES = 30  
GESTURE_CLASSES = sorted(os.listdir(DATASET_PATH))  # List gesture folders

# Initialize lists
X_landmarks = []
y = []

print("Detected gestures:", GESTURE_CLASSES)

# Process each gesture
for label, gesture in enumerate(GESTURE_CLASSES):
    gesture_folder = os.path.join(DATASET_PATH, gesture)
    
    # Detect landmark CSV dynamically
    landmark_csv_files = glob.glob(os.path.join(gesture_folder, "landmarks*.csv"))
    
    if not landmark_csv_files:
        print(f"⚠️ Warning: No landmarks CSV found for {gesture}, skipping...")
        continue
    
    landmark_csv_path = landmark_csv_files[0]
    df_landmarks = pd.read_csv(landmark_csv_path)

    # Reshape landmarks into (100 samples, 50 frames, landmark features)
    num_samples = len(df_landmarks) // NUM_FRAMES

    if num_samples < 100:
        print(f"⚠️ Warning: {gesture} has only {num_samples} samples (expected 100), skipping...")
        continue

    landmarks = df_landmarks.values.reshape(num_samples, NUM_FRAMES, -1)

    # Append data
    X_landmarks.extend(landmarks)
    y.extend([label] * num_samples)

# Convert to numpy arrays
X_landmarks = np.array(X_landmarks)
y = to_categorical(np.array(y), num_classes=len(GESTURE_CLASSES))

# Debugging print
print(f"✅ Total valid samples collected: {X_landmarks.shape[0]}")

# Proceed if data is available
if X_landmarks.shape[0] == 0:
    raise ValueError("❌ No valid samples found. Check dataset structure!")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_landmarks, y, test_size=0.2, random_state=42)

# Save preprocessed data
np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)

print("✅ Preprocessing completed successfully! Ready for 3D CNN training.")


Detected gestures: ['I am Inavatable', 'M Chuda', 'Time reversal', 'wave']
✅ Total valid samples collected: 400
✅ Preprocessing completed successfully! Ready for 3D CNN training.


LSTM Model for Dynamic Guesture Prediction

In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

# Improved LSTM Model
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(30, 63)),  # More units
    BatchNormalization(),
    #Dropout(0.5),

    LSTM(64, return_sequences=True),  # Reduce for better generalization
    BatchNormalization(),
    LSTM(32, return_sequences=False),
    #Dropout(0.5),

    Dense(32, activation='relu'),
    #Dropout(0.3),

    Dense(4, activation='softmax')  # Assuming 4 classes
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train1, y_train1, epochs=200, batch_size=16)
#model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))
model.save("gesture_lstm_model_v1.h5")
print("✅ Model saved successfully!")


Epoch 1/200


c:\Users\AYUSH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.2601 - loss: 1.4058
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2667 - loss: 1.3813
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2904 - loss: 1.3891
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2613 - loss: 1.3926
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2511 - loss: 1.3785
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4540 - loss: 1.3244
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2505 - loss: 1.3889
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2567 - loss: 1.4032
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2399 - loss: 1.3811
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2376 - loss: 1.3679
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3867 - loss: 1.2541
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy:

✅ Model saved successfully!


In [81]:
import numpy as np
def isStatic(landmark_history,threshold=10):
    if len(landmark_history)<2:
        return True
    distance=np.mean(np.linalg.norm(np.array(landmark_history[-1])-np.array(landmark_history[-2])))
    return distance<threshold

In [85]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow import keras
from collections import deque
from sklearn.preprocessing import LabelEncoder

# Load Static Gesture Model (CNN)
static_model = keras.models.load_model("gesture_model.keras")

# Load Dynamic Gesture Model (3D CNN)
dynamic_model = keras.models.load_model("gesture_lstm_model_v1.h5")

# Load Label Encoder for Gesture Labels
label_encoder = LabelEncoder()
label_classes = np.load("label_classes.npy", allow_pickle=True)
label_encoder.classes_ = label_classes  # Assign loaded labels

# Define Gesture Labels (Update according to your dataset)
STATIC_GESTURES = ["Fist", "fuck off", "Open Palm", "Spiderman", "Thumbs Up", "Thumbs Down", "Victory"]
DYNAMIC_GESTURES = ["I am Inavatable","M Chuda","Time reversal","wave"]

# Debug: Print loaded gestures
print("Loaded Static Gestures:", STATIC_GESTURES)
print("Loaded Dynamic Gestures:", DYNAMIC_GESTURES)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Store last 30 frames of landmarks
landmark_history = deque(maxlen=30)

# Function to detect if hand is static
def isStatic(landmark_history, threshold=0.1):  # Lowered threshold for better sensitivity
    """Check if hand is static by comparing last two frames."""
    if len(landmark_history) < 2:
        return True  # Assume static if not enough frames

    # Compute Euclidean distance between last two frames
    distance = np.linalg.norm(np.array(landmark_history[-1]) - np.array(landmark_history[-2]))

    return distance < threshold  # If movement is below threshold, it's static

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract landmarks (21 points, each with x, y, z)
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            # Store in history
            landmark_history.append(landmarks)

            # Check if static or dynamic
            if isStatic(landmark_history):
                if len(landmarks) == 63:  # Ensure correct shape
                    landmarks_array = np.array(landmarks).reshape(1, -1).astype(np.float32)

                    # Make prediction
                    prediction = static_model.predict(landmarks_array)
                    print("Raw model output:", prediction)  # Print raw model output

                    # Convert logits to probabilities (only if your model doesn't already output probabilities)
                    if prediction.shape[1] > 1:  # Check if multi-class
                        prediction_prob = keras.activations.softmax(keras.backend.variable(prediction)).numpy()
                    else:
                        prediction_prob = prediction  # If already probabilities

                    print("Softmax probabilities:", prediction_prob)  # Debug print

                    # Get predicted class index
                    predicted_index = np.argmax(prediction_prob)
                    print("Predicted index:", predicted_index)  # Debug print

                    # Ensure the index is valid
                    if 0 <= predicted_index < len(label_encoder.classes_):
                        gesture = label_encoder.inverse_transform([predicted_index])[0]
                    else:
                        gesture = "Unknown"

                    print("Predicted gesture:", gesture)  # Debug print


                    # Ensure valid index
                    if 0 <= predicted_index < len(label_encoder.classes_):
                        if(predicted_index==0):
                            gesture="fist"
                        if(predicted_index==1):
                            gesture="fuck off"
                        if(predicted_index==2):
                            gesture="open palm" 
                        if(predicted_index==3):
                            gesture="spiderman"
                        if(predicted_index==5):
                            gesture="thumbs upp"
                        if(predicted_index==6):
                            gesture="thumbs down"
                        if(predicted_index==6):
                            gesture="victory"  
                        #gesture = label_encoder.inverse_transform([predicted_index])[0]
                    else:
                        gesture = "thumbs down"

                    # Display static gesture
                    cv2.putText(frame, "Static", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(frame, f"Gesture: {gesture}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "Error: Landmark Shape", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                # Check if buffer is full for dynamic gesture classification
                if len(landmark_history) == FRAME_BUFFER_SIZE:
                    landmark_array = np.array(landmark_history)  # Shape (30, 63)
                    '''
                    # Add a column of zeros to make it (30, 64)
                    extra_feature = np.zeros((FRAME_BUFFER_SIZE, 1))  
                    landmark_array = np.hstack((landmark_array, extra_feature))  # Now shape (30, 64)'''

                    dynamic_input = landmark_array.reshape(1, 30, 63)  # Ensure (1, 30, 64)
                    dynamic_prediction = dynamic_model.predict(dynamic_input)
                    dynamic_index = np.argmax(dynamic_prediction)


                    if 0 <= dynamic_index < len(DYNAMIC_GESTURES):
                        dynamic_gesture = DYNAMIC_GESTURES[dynamic_index]
                    else:
                        dynamic_gesture = "Unknown"


                    # Display Dynamic Gesture
                    cv2.putText(frame, "Dynamic Gesture", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(frame, f"Gesture: {dynamic_gesture}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Hand Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()


Loaded Static Gestures: ['Fist', 'fuck off', 'Open Palm', 'Spiderman', 'Thumbs Up', 'Thumbs Down', 'Victory']
Loaded Dynamic Gestures: ['I am Inavatable', 'M Chuda', 'Time reversal', 'wave']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Raw model output: [[7.93023573e-05 5.64963790e-03 9.79823172e-01 2.25909078e-03
  7.06109358e-06 1.79180563e-06 1.21799745e-02]]
Softmax probabilities: [[0.11516006 0.11580332 0.30676034 0.11541136 0.11515173 0.11515113
  0.11656203]]
Predicted index: 2
Predicted gesture: -0.2151748538017273
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Raw model output: [[2.4364707e-04 5.8514434e-03 9.5542049e-01 4.9378690e-03 7.8464054e-06
  4.0404166e-06 3.3534653e-02]]
Softmax probabilities: [[0.11570279 0.11635344 0.3007266  0.11624719 0.11567551 0.11567506
  0.11961947]]
Predicted index: 2
Predicted gesture: -0.2151748538017273
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Raw model output: [[1.1349453e-04 1.3662164e-02 6.5202224e-01 1.6466517e-02 3.1277654e-05
  6.9309467e-06 3.1769735